In [1]:
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
import torch
import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
import json

In [4]:
#!rm *txt
#!rm *jsonl


In [2]:
EPOCH_NUM = 1000
TEST_NUM = 2000
TRAIN_NUM = 16 # training dataset size
POLY_NUM = 3
D = 4 # number of domains
HEAD_NUM = 8 #D * 8
RUN_NUM = 20
GRAD_CLIP = 1500.0 # for unstable gradients pruning

def rmse(y, y_hat):
    return torch.sqrt(((y-y_hat)**2).mean())
crit2 = rmse # eval criterion

In [3]:
def make_angles():
    if D == 2:
        return [0, 180] # hard-coding angles for 2D cases (to be opposite)
    step = 360/D
    return [i*step for i in range(D)]
make_angles()

[0.0, 90.0, 180.0, 270.0]

In [4]:
# data generation
from scipy.ndimage import rotate
def gen(N1, N2, sigma=.1, seed=42, angle = 0.0):
    """
    N1: train size
    N2: test size
    """
    angle = np.pi/2 * angle/90
    N = N1+N2
    rs = np.random.RandomState(seed)
    x = rs.randn(N, 2) 
    y = x[:,0]*0.25 + x[:,1]*0.75 # y = X1 * 0.25 + x2 * 0.75, other  X components are irrelevant 
    
    if angle: # making rotation if angle != 0
        M = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        x[:,:2] = (M@x[:,:2].T).T
    # shuffling
    elems = list(range(len(x)))
    rs.shuffle(elems)
    trainval, test = elems[:N1], elems[N1:]
    xtrainval, ytrainval = x[trainval], y[trainval]
    xtest,ytest = x[test], y[test]
    elems = list(range(len(xtrainval)))
    rs.shuffle(elems)
    # actually this code is a bit messy, it was inherited from classification task
    train, val = elems[:N1//2], elems[N1//2:]
    
    xtrain, ytrain = xtrainval[train], ytrainval[train]
    xval, yval = xtrainval[val], ytrainval[val]
    return (xtrain, ytrain), (xval, yval), (xtest, ytest)
xy, _,_ = gen(4, 2, angle=0)


In [5]:
def make_inference(x, rots, models, gammas, use_softmax=True, only_angle=False, c_head = False, t=1.0):
    
    if use_softmax:
        g0 = torch.distributions.RelaxedOneHotCategorical(t, logits=gammas[0]).rsample((x.shape[0],))
        x = torch.sum(torch.cat([(r(x)*g0[:, i].view(-1, 1)).unsqueeze(2) for i, r in enumerate(rots)], 2), 2)
    
    else:
        if len(gammas[0].shape) == 1:
            g0 = (gammas[0] == gammas[0].max()) * 1
            x = torch.sum(torch.cat([(r(x)*g0[i]).unsqueeze(2) for i, r in enumerate(rots)], 2), 2)
        else: # sample from rl
            g0 = gammas[0]
            x = torch.sum(torch.cat([(r(x)*g0[:, i].view(-1, 1)).unsqueeze(2) for i, r in enumerate(rots)], 2), 2)
    
    
    if only_angle:
        return x 
    x = poly_generate(x)
    if c_head:
        g1 = torch.zeros(gammas[1].shape)
        g1[0] += 0
        
    if use_softmax:
        g1 = torch.distributions.RelaxedOneHotCategorical(t, logits=gammas[1]).rsample((x.shape[0],))
        x = torch.sum(torch.cat([(m(x)*g1[:, i].view(-1, 1)).unsqueeze(2) for i, m in enumerate(models)], 2), 2)
       
    else:
        if len(gammas[0].shape) == 1:
            g1 = (gammas[1] == gammas[1].max()) * 1
            x = torch.sum(torch.cat([(m(x)*g1[i]).unsqueeze(2) for i, m in enumerate(models)], 2), 2)
        else: # sample from rl
            g1 = gammas[1]
            x = torch.sum(torch.cat([(m(x)*g1[:, i].view(-1, 1)).unsqueeze(2) for i, m in enumerate(models)], 2), 2)
    
    return x



In [21]:
xy[0][0]

array([-0.46341769, -0.46572975])

In [6]:
def poly_generate(x):
    result = [x]
    for poly in range(POLY_NUM):
        deg = poly+2 
        new_x = x ** deg#/(deg)
        result.append(new_x)
    result =  torch.hstack(result)
    return result
poly_generate(torch.tensor(xy[0])).shape

torch.Size([2, 8])

In [7]:
# one-model for strictly one domain, no NAS
accs = []
for k in range(RUN_NUM):
    torch.manual_seed(42+k)
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = gen(TRAIN_NUM, TEST_NUM, seed=42+k)
    x_train = np.concatenate([x_train, x_val]) # we don't use train/validation step
    y_train = np.concatenate([y_train, y_val])
    
    lin_model = torch.nn.Linear(2*(POLY_NUM+1),1)
    model = lambda x: lin_model(poly_generate(x))
    opt = torch.optim.SGD(lin_model.parameters(), lr=1e-3)
    crit = torch.nn.MSELoss()
    tq = tqdm.tqdm_notebook(range(EPOCH_NUM))
    
    for e in tq:
        opt.zero_grad()
        out = model(torch.tensor(x_train).float())[:,0]
        loss = crit(out, torch.tensor(y_train).float())
        loss.backward()
        torch.nn.utils.clip_grad_value_(lin_model.parameters(), GRAD_CLIP)
        opt.step()
        tq.set_description(str(loss.item()))
    acc = crit2(model(torch.tensor(x_test).float())[:,0], torch.tensor(y_test).float()).item()
    accs.append(acc)
    
print (accs, np.mean(accs)) 

with open('one_model.txt', 'a') as out:
    out.write(f'D={D}, HEAD_NUM={HEAD_NUM}, RUN_NUM={RUN_NUM}, CRIT={np.mean(accs)}+-{np.std(accs)}.   MIN={np.min(accs)}, MAX={np.max(accs)}\n')
with open('one_model.jsonl', 'a') as out:
    out.write(json.dumps({'D': D, 'HEAD_NUM':HEAD_NUM, 'RUN_NUM': RUN_NUM, 'accs': accs})+'\n')
    

<ipython-input-7-da2e9f739812>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM))


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [35]:
# shared-model:
# one-model for all the domains, without any rotation, without NAS
accs = []
for k in range(RUN_NUM):
    torch.manual_seed(42+k)
    x_train, y_train, x_test, y_test = [],[],[],[]
    for angle in make_angles():
        (_x_train, _y_train), (x_val, y_val), (_x_test, _y_test) = gen(TRAIN_NUM,
                                                                       TEST_NUM, seed=42+k+int(angle),
                                                                       angle=angle)
        
            
        
        _x_train = np.concatenate([_x_train, x_val]) # we don't use train/validation step
        _y_train = np.concatenate([_y_train, y_val])
        x_train.extend(_x_train)
        y_train.extend(_y_train)
        x_test.extend(_x_test)
        y_test.extend(_y_test)
    
    lin_model = torch.nn.Linear(2*(1+POLY_NUM),1) 
    opt = torch.optim.SGD(lin_model.parameters(), lr=1e-3)
    model = lambda x: lin_model(poly_generate(x))
    crit = torch.nn.MSELoss()
    tq = tqdm.tqdm_notebook(range(EPOCH_NUM  * D)) # in darts we make D iterations per epoch. So here we multiply EPOCH_NUM  * D for compensation
    
    for e in tq:
        opt.zero_grad()
        out = model(torch.tensor(x_train).float())[:,0]
        loss = crit(out, torch.tensor(y_train).float())
        loss.backward()
        torch.nn.utils.clip_grad_value_(lin_model.parameters(), GRAD_CLIP)
        opt.step()
        tq.set_description(str(loss.item()))
    acc = crit2(model(torch.tensor(x_test).float())[:,0], torch.tensor(y_test).float()).item()
    accs.append(acc)
    print (acc)
print (accs, np.mean(accs)) 

with open('shared_model.txt', 'a') as out:
    out.write(f'D={D}, HEAD_NUM={HEAD_NUM}, RUN_NUM={RUN_NUM},  CRIT={np.mean(accs)}+-{np.std(accs)}. MIN={np.min(accs)}, MAX={np.max(accs)}\n')
with open('shared_model.jsonl', 'a') as out:
    out.write(json.dumps({'D': D, 'HEAD_NUM':HEAD_NUM, 'RUN_NUM': RUN_NUM, 'accs': accs})+'\n')

<ipython-input-35-944c4b619fd1>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM  * D)) # in darts we make D iterations per epoch. So here we multiply EPOCH_NUM  * D for compensation


  0%|          | 0/4000 [00:00<?, ?it/s]

1.3885122537612915


  0%|          | 0/4000 [00:00<?, ?it/s]

1.4878994226455688


...

0.8789204359054565


  0%|          | 0/4000 [00:00<?, ?it/s]

1.004591941833496


  0%|          | 0/4000 [00:00<?, ?it/s]

0.8532770276069641
[1.3885122537612915, 1.4878994226455688, 0.8789204359054565, 1.004591941833496, 0.8532770276069641] 1.1226402163505553


In [ ]:
lin_model.weight

In [9]:
# perfect-case:
# one model for all the data, but the data is perfectly rotated
accs = []
for k in range(RUN_NUM):
    torch.manual_seed(42+k)
    x_train, y_train, x_test, y_test = [],[],[],[]
    for angle in make_angles():
        (_x_train, _y_train), (x_val, y_val), (_x_test, _y_test) = gen(TRAIN_NUM, 2000, seed=42+k+int(angle),
                                                                       angle=0.0)
        
        _x_train = np.concatenate([_x_train, x_val]) # we don't use train/validation step
        _y_train = np.concatenate([_y_train, y_val])
        x_train.extend(_x_train)
        y_train.extend(_y_train)
        x_test.extend(_x_test)
        y_test.extend(_y_test)
    
    
    lin_model = torch.nn.Linear(2*(POLY_NUM+1),1) 
    opt = torch.optim.SGD(lin_model.parameters(), lr=1e-3)
    model = lambda x: lin_model(poly_generate(x))
    crit = torch.nn.MSELoss()
    tq = tqdm.tqdm_notebook(range(EPOCH_NUM * D))
    
    for e in tq:
        opt.zero_grad()
        out = model(torch.tensor(x_train).float())[:,0]
        loss = crit(out, torch.tensor(y_train).float())
        loss.backward()
        torch.nn.utils.clip_grad_value_(lin_model.parameters(), GRAD_CLIP)
        opt.step()
        tq.set_description(str(loss.item()))
    acc = crit2(model(torch.tensor(x_test).float())[:,0], torch.tensor(y_test).float()).item()
    accs.append(acc)
    print (acc)
print(np.mean(accs)) 
with open('perfect_model.txt', 'a') as out:
    out.write(f'D={D}, HEAD_NUM={HEAD_NUM}, RUN_NUM={RUN_NUM}, CRIT={np.mean(accs)}+-{np.std(accs)}.   MIN={np.min(accs)}, MAX={np.max(accs)}\n')
with open('perfect_model.jsonl', 'a') as out:
    out.write(json.dumps({'D': D, 'HEAD_NUM':HEAD_NUM, 'RUN_NUM': RUN_NUM, 'accs': accs})+'\n')

<ipython-input-9-3fdedc7601c8>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM * D))


  0%|          | 0/4000 [00:00<?, ?it/s]

<ipython-input-9-3fdedc7601c8>:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  out = model(torch.tensor(x_train).float())[:,0]


0.0724780336022377


  0%|          | 0/4000 [00:00<?, ?it/s]

...

0.1256198137998581


  0%|          | 0/4000 [00:00<?, ?it/s]

0.3906998634338379


  0%|          | 0/4000 [00:00<?, ?it/s]

0.5258259773254395
0.2479586862027645


In [8]:
# rotation module for torch. Angles are in degrees
class Rot(torch.nn.Module):
    def __init__(self, angle = None):
        super().__init__()
        if angle is None:
            angle = torch.randn(1)*90
        else:
            angle = torch.tensor(angle).float()
        self.angle = torch.nn.Parameter(angle)
    
    def forward(self, x):
        angle = self.angle * np.pi/2 / 90
        c = torch.cos(angle)
        s = torch.sin(angle)
        M = torch.vstack([torch.hstack([c, -s]), torch.hstack([s, c])])
        result = x * 1.0
        result[:,:2] = (M@x[:,:2].T).T
        return result


In [9]:
nn = torch.nn
# simple triplet loss: for each objects takes a random triplet (with triplet conditions)
class MdTripletLoss(nn.Module):
    def __init__(self, m =0.0, p=2, subset_size = 9999, sample_num = 1):
        super(MdTripletLoss, self).__init__()
        self.triplet_loss = nn.TripletMarginLoss(margin=m, p=p)
        self.rs = np.random.RandomState(21)
        self.subset_size = subset_size
        self.sample_num = sample_num
    
    def forward(self, h1: torch.Tensor, h2: torch.Tensor, labels1: torch.LongTensor, labels2: torch.LongTensor):
        """
        :param: h1: hidden representations of size (bs, *), anchors
        :param: h1: hidden representations of size (bs, *), positives and negatives candidates
        """
        FEATURE_TRIPLET_DIM = 2
        h1 = h1[:, :FEATURE_TRIPLET_DIM]
        h2 = h2[:, :FEATURE_TRIPLET_DIM]
        
        bs = h1.size(0)
        if bs > self.subset_size:
            elems = list(range(bs))
            self.rs.shuffle(elems)
            h1 = h1[elems[:self.subset_size]]
            labels1 = labels1[elems[:self.subset_size]]
            self.rs.shuffle(elems)
            h2 = h2[elems[:self.subset_size]]
            labels2 = labels2[elems[:self.subset_size]]
            bs = self.subset_size
            
        
        h1 = h1.view(bs, -1)
        h2 = h2.view(bs, -1)
        
        loss = 0.0
        anch =  []
        pos = []
        neg = []
        ids1 = list(range(len(h1)))
        
        ids2 = list(range(len(h2)))
        rs = self.rs
        for _ in range(self.sample_num):
           
            #ids2 = ids2[::-1]
            rs.shuffle(ids1)
            rs.shuffle(ids2)
            h1 = h1[ids1]
            h2 = h2[ids2]
            labels1 = labels1[ids1]
            labels2 = labels2[ids2]

            for i in range(len(h1)):
                found = False 
                for j in range(len(h2)):

                    if found:
                        break
                    for k in range(j+1, len(h2)):
                        if   abs(labels1[i] - labels2[j]) > abs(labels1[i] - labels2[k]): #\
                        #and torch.linalg.norm(h1[i] - h2[j]) < torch.linalg.norm(h1[i] - h2[k]):
                            anch.append(h1[i])
                            pos.append(h2[k])
                            neg.append(h2[j])

                            found = True
                            break
        
        if len(anch) >0:
            a = torch.vstack(anch)
            p = torch.vstack(pos)
            n = torch.vstack(neg)
            loss = self.triplet_loss(a, p, n)
            return loss
        else:
            return 0.0

In [32]:
#model = ArchModel()
#len(list((model.models[0].parameters()))[0][0])


9

In [10]:
# our architecture
# consists of HEAD_NUM rotations and HEAD_NUN linear models

class ArchModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.gammas = [torch.nn.Parameter(torch.rand(2, HEAD_NUM)) for _ in range(D)]
        self.rots = torch.nn.ModuleList([Rot() for _ in range(HEAD_NUM)])
        self.models = torch.nn.ModuleList([torch.nn.Linear(2*(POLY_NUM+1), 1) for _ in range(HEAD_NUM)])
    def forward(self, x, d, gammas = None, only_angle = False, use_softmax = True, t=1.0):
        if gammas is None:
            gammas = self.gammas[d]
        return make_inference(x, self.rots, self.models, self.gammas[d],
                              use_softmax=use_softmax, only_angle=only_angle, t=t)
   

def multi_js_divergence(alphas) -> torch.tensor:
    js = 0.0
    for j in range(2): # rotation and head
        alpha_full = torch.stack([a[j] for a in alphas], dim=0)
        alpha_full = torch.softmax(alpha_full, dim=1)
        
        centroid = torch.mean(alpha_full, 0)
        
        d_centroid = torch.distributions.Categorical(probs=centroid)
        
        for alpha in alphas:
            d = torch.distributions.Categorical(probs=torch.softmax(alpha[j], 0))
            js += 1.0/len(alphas) * \
                torch.distributions.kl.kl_divergence(d, d_centroid)
    return js

# param number calculation   
def calc_params(gammas):
    linear_model_param_num = 2* ( POLY_NUM + 1) + 1
    cnt = 0
    rots = set()
    for i in range(D):
        rots.add(gammas[i][0].argmax().item())
    
    linears = set()
    for i in range(D):
        linears.add(gammas[i][1].argmax().item())
    return len(rots) + len(linears)*(linear_model_param_num)

#gammas = [torch.randn(2, 5) for _ in range(3)]
#multi_js_divergence(gammas)

In [11]:
def dartslike(fname, triplet_coef_search = 0.0, triplet_coef_train = 0.0, js_coef = 0.0, unroll: bool = True,
              lr=1e-3, lr0=0.1, lr2=.01):
    h = []
    accs = []
    param_num = []

    
    tl = MdTripletLoss(p=2.0, subset_size=9999, sample_num=1) # oleg
    
    for k in range(RUN_NUM):
        torch.manual_seed(42+k)
        x_train, y_train, x_val, y_val, x_test, y_test = [],[],[],[],[],[]
        for angle in make_angles():
            (_x_train, _y_train), (_x_val, _y_val), (_x_test, _y_test) = gen(TRAIN_NUM, TEST_NUM, seed=42+k+int(angle),
                                                                             angle=angle)
            x_train.append(_x_train)
            y_train.append(_y_train)
            x_val.append(_x_val)
            y_val.append(_y_val)
            x_test.append(_x_test)
            y_test.append(_y_test)
        model = ArchModel()
        
    
        rs = np.random.RandomState(42+k)
        
        opt = torch.optim.SGD([{'params': model.rots.parameters(), 'lr': lr0},
                              {'params': model.models.parameters(), 'lr': lr}],
                              lr=lr)
                              
                              
        opt2 = torch.optim.SGD(model.gammas, lr=lr2)

        crit = torch.nn.MSELoss()
        tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS
        losses = []
        losses2 = []
        for e in tq:
            t = max(0.2, 1.0 - e/EPOCH_NUM)
            
            for d in range(D):
            
                opt2.zero_grad()
                d2 = rs.choice([i for i in range(D) if i != d])
            
                # unrolling step: done manually without approximation
                if unroll:
                    opt.zero_grad()
                    x = torch.tensor(x_train[d]).float()
                    y = torch.tensor(y_train[d]).float()

                    x2 = torch.tensor(x_train[d2]).float()
                    y2 = torch.tensor(y_train[d2]).float()

                    out_angle1 = model(x, d, only_angle=True, t=t)
                    out_angle2 = model(x2, d2, only_angle=True, t=t)

                    out = model(x, d, t=t)[:,0]
                    loss = crit(out, y)
                    if triplet_coef_search:
                        tloss = tl(out_angle1, out_angle2, y, y2) * triplet_coef_search
                        loss += tloss
                    grads = torch.autograd.grad(loss, model.parameters())
                    grads = [torch.clamp(g, -GRAD_CLIP, GRAD_CLIP) for g in grads]
                        
                    with torch.no_grad():
                        for p,g in zip(model.parameters(), grads):
                            p = p + lr * g
                            
                x = torch.tensor(x_val[d]).float()
                y = torch.tensor(y_val[d]).float()

                x2 = torch.tensor(x_val[d2]).float()
                y2 = torch.tensor(y_val[d2]).float()

                out_angle1 = model(x, d, only_angle=True, t=t)
                out_angle2 = model(x2, d2, only_angle=True, t=t)
                

                out = model(x, d, t=t)[:,0]
                loss2 = crit(out, y) 
                if triplet_coef_search:
                        tloss = tl(out_angle1, out_angle2, y, y2) * triplet_coef_search
                        loss2 += tloss
                if js_coef:
                    loss2 += multi_js_divergence(model.gammas).mean() * js_coef
                loss2.backward()
                torch.nn.utils.clip_grad_value_(model.gammas, GRAD_CLIP)
                opt2.step()
                
                if unroll:
                    with torch.no_grad():
                        for i,g in zip(model.parameters(), grads):
                            p = p - lr * g
                            
                            
                opt.zero_grad()
                x = torch.tensor(x_train[d]).float()
                y = torch.tensor(y_train[d]).float()

                x2 = torch.tensor(x_train[d2]).float()
                y2 = torch.tensor(y_train[d2]).float()

                out_angle1 = model(x, d, only_angle=True, t=t)
                out_angle2 = model(x2, d2, only_angle=True, t=t)
                
                out = model(x, d)[:,0]
                loss = crit(out, y)
                if triplet_coef_search:
                    tloss = tl(out_angle1, out_angle2, y, y2) * triplet_coef_search
                    loss += tloss
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP)
                opt.step()
                losses.append(loss.item())
                losses2.append(loss2.item())
            
            tq.set_description(str(np.mean(losses))+';'+str(np.mean(losses2)))

        tq = tqdm.tqdm_notebook(range(EPOCH_NUM))
        
        # search step. Now we concatenate train and validation
        x_train = np.concatenate([x_train, x_val], axis=1)
        y_train =  np.concatenate([y_train, y_val], axis=1)
        
        
        for g in model.gammas:
            print (g.argmax(1))
        
        for g in model.gammas:
            print (g)
        
        opt = torch.optim.SGD([{'params': model.rots.parameters(), 'lr': lr0},
                              {'params': model.models.parameters(), 'lr': lr}],
                              lr=lr)
        
        for e in tq:
            
            for d in range(D):
                loss = 0.0
                opt.zero_grad()

                d2 = rs.choice([i for i in range(D) if i != d])

                x = torch.tensor(x_train[d]).float()
                y = torch.tensor(y_train[d]).float()
                out = model(x, d, use_softmax=False)[:,0]

               
                loss = crit(out, y) 
                if triplet_coef_train:
                    x2 = torch.tensor(x_train[d2]).float()
                    y2 = torch.tensor(y_train[d2]).float()

                    out_angle1 = model(x, d, only_angle=True, use_softmax=False) # note, using one-hot here
                    out_angle2 = model(x2, d2, only_angle=True, use_softmax=False)

                    tloss = tl(out_angle1, out_angle2, y, y2)* triplet_coef_train
                    loss += tloss
                
                
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP)
                opt.step()
            angles= [str(r.angle.item()) for r in model.rots]
            #print (angles)
            tq.set_description(' '.join(angles))
        total = 0
        for d in range(D):

            out = model(torch.tensor(x_test[d]).float(), d, use_softmax=False)
            total += crit2(out[:,0], torch.tensor(y_test[d]).float()).item()
            print ('acc ', d, crit2(out[:,0], torch.tensor(y_test[d]).float()).item())
        acc = total/D
        accs.append(acc)
        param_num.append(calc_params(model.gammas))
        print (acc, param_num[-1])
        
    with open(fname+'.txt', 'a') as out:
        out.write(f'D={D}, HEAD_NUM={HEAD_NUM}, RUN_NUM={RUN_NUM}, CRIT={np.mean(accs)}+-{np.std(accs)}.  MIN={np.min(accs)}, MAX={np.max(accs)}')
        out.write(f' PARAMS={np.mean(param_num)}+-{np.std(param_num)}. MIN={np.min(param_num)}, MAX={np.max(param_num)}\n')
    
    with open(fname+'.jsonl', 'a') as out:
        out.write(json.dumps({'D': D, 'HEAD_NUM': HEAD_NUM, 'RUN_NUM': RUN_NUM, 'accs': accs, 'params': param_num})+'\n')
    print (model.gammas)
    for r in model.rots:
        print (r.angle)
    return (np.mean(accs), np.mean(param_num))


In [12]:
# helper for make inference for a model

#for D in [2,3,4]:
#    for HEAD_NUM in [D, 8]:
for D in [4]:
    for HEAD_NUM in [8]:

        dartslike('triplets_gs', 1000.0, 1000.0)

<ipython-input-9-7736bc93d887>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS


  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-9-7736bc93d887>:120: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM))


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([6, 0])
tensor([0, 6])
tensor([3, 6])
tensor([1, 0])
Parameter containing:
tensor([[-0.0225, -3.1178, -1.4113, -0.3959,  0.9707,  1.3132,  9.2108, -1.3661],
        [ 0.9071,  0.2940,  0.6892,  0.6719,  0.6382,  0.7046,  0.7425,  0.5624]],
       requires_grad=True)
Parameter containing:
tensor([[ 1.0739e+01, -1.6952e+00,  6.3089e-03, -2.4048e+00,  5.2030e-01,
          1.9893e+00, -1.7392e+00, -3.2227e+00],
        [ 1.3185e-01,  2.5738e-01,  1.3150e-01,  6.5252e-02,  6.4609e-01,
         -9.4677e-02,  1.1486e+00,  2.2714e-01]], requires_grad=True)
Parameter containing:
tensor([[-8.1441e-01, -3.5143e-01,  4.1263e-01,  1.0004e+01, -2.1638e+00,
         -1.4262e+00,  1.2386e-01, -7.9007e-01],
        [ 8.3684e-01,  8.5815e-01,  1.0491e+00,  7.3535e-01,  1.0527e+00,
          4.4803e-03,  1.0565e+00, -4.8895e-01]], requires_grad=True)
Parameter containing:
tensor([[ 1.4904, 11.1621, -2.8820, -2.3336, -0.7658, -1.1921, -0.9870, -0.8530],
        [ 1.9639,  0.6081,  1.2531, -0.0895,

...

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([6, 1])
tensor([4, 7])
tensor([1, 0])
tensor([0, 1])
Parameter containing:
tensor([[ 0.8196, -1.5378, -2.0365,  0.0433, -1.4232, -1.5676, 11.4848, -1.5119],
        [ 0.5362,  1.6306,  1.0948,  0.1645,  0.2557, -0.3713,  1.3761, -0.7070]],
       requires_grad=True)
Parameter containing:
tensor([[-1.2612,  0.7653, -1.7499,  1.1703, 11.5981,  0.6716, -3.3347, -3.2835],
        [ 0.4290,  0.7219,  0.1211, -0.0492,  0.6288,  0.0428,  0.1842,  0.7867]],
       requires_grad=True)
Parameter containing:
tensor([[-2.0845, 10.0749, -0.7338, -0.9913, -1.6352, -0.2901, -0.7865,  0.6639],
        [ 1.4981,  1.2533,  0.3771,  0.8232,  0.8809,  0.0705,  1.3597, -0.2719]],
       requires_grad=True)
Parameter containing:
tensor([[11.6024, -1.9473, -0.7191, -3.4519, -1.4556, -2.3969,  0.0226,  2.0182],
        [ 0.6921,  0.8319,  0.8190, -0.1846,  0.2523,  0.1244,  0.3609, -0.0652]],
       requires_grad=True)
acc  0 0.6464376449584961
acc  1 1.7932721376419067
acc  2 0.5077188014984131
acc  3

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([0, 2])
tensor([1, 6])
tensor([5, 5])
tensor([0, 1])
Parameter containing:
tensor([[10.6168, -0.5528, -0.6010, -1.0621, -1.2049, -1.7602,  0.4434, -0.7555],
        [ 0.0508,  0.5931,  1.0343,  0.8964,  0.5063,  0.6903, -0.0740,  0.1210]],
       requires_grad=True)
Parameter containing:
tensor([[-2.2080, 13.1822, -1.6983, -0.7355, -1.7869,  0.2776, -1.8081, -0.9574],
        [ 0.5310,  0.5460,  0.1802, -0.3401,  0.8060,  0.4814,  2.3477,  0.5110]],
       requires_grad=True)
Parameter containing:
tensor([[-0.1738, -0.5638, -0.4170, -2.0826, -1.5136, 11.8069, -4.6158,  2.3615],
        [ 0.1926,  0.7038,  0.1911,  0.0407,  0.4767,  1.0140,  0.0431,  0.6654]],
       requires_grad=True)
Parameter containing:
tensor([[ 9.0580, -3.7388,  0.8577, -0.5009, -1.9497,  0.2266, -1.3708,  0.3624],
        [ 0.6263,  0.8620,  0.7798,  0.3756,  0.6927,  0.0956,  0.7488,  0.5942]],
       requires_grad=True)
acc  0 0.7519489526748657
acc  1 0.4081135094165802
acc  2 0.5345773696899414
acc  3

In [12]:
# no reg
for D in [2,3,4]:
    for HEAD_NUM in [D, 8]:
        dartslike('darts', 0.0, 0.0)


<ipython-input-11-7736bc93d887>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS


  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-11-7736bc93d887>:120: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM))


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([0, 0])
tensor([0, 1])
Parameter containing:
tensor([[1.0506, 0.7466],
        [1.2429, 0.0993]], requires_grad=True)
Parameter containing:
tensor([[ 1.2469, -0.2555],
        [-0.1186,  1.1688]], requires_grad=True)
acc  0 1.224833607673645
acc  1 1.0574719905853271
1.141152799129486 19


...

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([4, 3])
tensor([3, 0])
tensor([7, 2])
tensor([2, 5])
Parameter containing:
tensor([[ 0.1067,  0.2885,  0.7378,  0.2928,  1.0190,  0.5048,  0.6174,  0.7038],
        [ 0.2220,  0.3623,  0.9824,  1.1735, -0.3524,  0.2470,  0.7927,  0.5522]],
       requires_grad=True)
Parameter containing:
tensor([[0.8528, 0.0893, 0.1744, 1.0110, 0.2851, 0.6588, 0.8447, 0.6600],
        [0.7796, 0.3680, 0.1076, 0.0693, 0.4302, 0.2653, 0.2448, 0.6005]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.0316,  0.6974,  0.2515,  0.2455, -0.8037,  1.6938, -0.3531,  2.4543],
        [ 0.0691, -0.1780,  1.8285,  0.9487,  0.3434,  0.7392,  0.8259,  1.4141]],
       requires_grad=True)
Parameter containing:
tensor([[-0.2518,  0.1528,  2.4803, -0.2280, -0.1724,  1.3135,  0.1731,  0.2050],
        [-0.3929, -1.2180,  1.1780,  0.5724,  1.1811,  1.5087,  0.5006, -0.4990]],
       requires_grad=True)
acc  0 1.793028712272644
acc  1 0.5049993991851807
acc  2 1.1479054689407349
acc  3 0.46377471089363

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([6, 2])
tensor([1, 7])
tensor([2, 5])
tensor([7, 6])
Parameter containing:
tensor([[0.1139, 0.8137, 0.4118, 0.6686, 0.5169, 0.8629, 0.9500, 0.7859],
        [0.0606, 0.5180, 1.3921, 0.7091, 0.3683, 0.3627, 0.1658, 0.2416]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.5091,  2.0115,  0.0315,  0.7188, -0.0819,  0.3521,  0.6578,  0.0666],
        [ 0.3096,  0.7953,  0.7739,  0.4762,  0.9086,  0.0178,  0.2515,  1.5301]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.9125,  0.6101,  1.1920,  0.0837,  0.0559,  0.7823,  0.1593,  1.0060],
        [ 0.4031,  0.3755,  0.1208,  0.0274,  0.5836,  1.1496, -0.1402,  0.8077]],
       requires_grad=True)
Parameter containing:
tensor([[0.4559, 0.4497, 0.4417, 0.2019, 0.2386, 0.1871, 0.1759, 0.7937],
        [0.8111, 0.7355, 0.5442, 0.1494, 0.6445, 0.2975, 0.9739, 0.6189]],
       requires_grad=True)
acc  0 1.9382646083831787
acc  1 0.7755471467971802
acc  2 0.43928107619285583
acc  3 0.900898277759552
1.0134977772

In [13]:
for D in [2,3,4]:
    for HEAD_NUM in [D, 8]:
        dartslike('js', js_coef=100.0)

<ipython-input-11-7736bc93d887>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS


  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-11-7736bc93d887>:120: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM))


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([0, 0])
tensor([0, 0])
Parameter containing:
tensor([[1.3774, 0.4199],
        [0.6922, 0.6499]], requires_grad=True)
Parameter containing:
tensor([[0.9782, 0.0131],
        [0.5450, 0.5052]], requires_grad=True)
acc  0 0.807583212852478
acc  1 1.200205683708191
1.0038944482803345 10


...

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([7, 2])
tensor([7, 2])
tensor([7, 2])
tensor([7, 2])
Parameter containing:
tensor([[ 0.2782,  0.3852,  0.8200,  0.1958, -0.4220,  1.4225,  0.1219,  1.4692],
        [-0.0226, -0.3431,  1.4425,  0.8254,  0.2202, -0.0180,  0.7402,  1.1352]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.2991,  0.4241,  0.8683,  0.2361, -0.3929,  1.4609,  0.1759,  1.5045],
        [-0.1607, -0.4800,  1.2842,  0.6778,  0.0602, -0.0934,  0.5862,  0.9910]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.2654,  0.4235,  0.8018,  0.1933, -0.4999,  1.4192,  0.1326,  1.4814],
        [ 0.2231, -0.0742,  1.6636,  1.0642,  0.4414,  0.3033,  0.9806,  1.3889]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.2057,  0.3114,  0.7319,  0.1224, -0.5010,  1.3564,  0.0676,  1.3781],
        [-0.1764, -0.4722,  1.2863,  0.6731,  0.0621, -0.1031,  0.5805,  0.9806]],
       requires_grad=True)
acc  0 0.9519301056861877
acc  1 0.9958717226982117
acc  2 0.738353431224823
acc  3 

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([1, 2])
tensor([1, 2])
tensor([1, 2])
tensor([1, 2])
Parameter containing:
tensor([[0.7095, 0.9778, 0.6214, 0.5213, 0.3035, 0.6579, 0.6041, 0.7281],
        [0.2695, 0.6042, 0.7235, 0.3472, 0.6565, 0.2977, 0.2261, 0.6934]],
       requires_grad=True)
Parameter containing:
tensor([[0.5924, 0.8639, 0.5130, 0.4060, 0.1992, 0.5542, 0.5058, 0.6312],
        [0.4211, 0.7662, 0.8747, 0.5061, 0.8222, 0.4535, 0.3712, 0.8480]],
       requires_grad=True)
Parameter containing:
tensor([[0.6672, 0.9300, 0.5794, 0.4891, 0.2677, 0.6215, 0.5584, 0.6885],
        [0.2092, 0.5429, 0.6611, 0.2875, 0.5985, 0.2371, 0.1603, 0.6308]],
       requires_grad=True)
Parameter containing:
tensor([[0.4348, 0.6940, 0.3483, 0.2509, 0.0381, 0.3845, 0.3332, 0.4607],
        [0.3902, 0.7257, 0.8454, 0.4689, 0.7781, 0.4183, 0.3343, 0.8143]],
       requires_grad=True)
acc  0 1.2431328296661377
acc  1 0.8567472100257874
acc  2 0.6469011306762695
acc  3 1.1760374307632446
0.9807046502828598 10
[Parameter containing:

In [14]:
def rl(fname, lr=1e-3, lr0=0.1, lr2=.01):
    h = []
    accs = []
    param_num = []

    
    for k in range(RUN_NUM):
        torch.manual_seed(42+k)
        x_train, y_train, x_val, y_val, x_test, y_test = [],[],[],[],[],[]
        for angle in make_angles():
            (_x_train, _y_train), (_x_val, _y_val), (_x_test, _y_test) = gen(TRAIN_NUM, TEST_NUM, seed=42+k+int(angle),
                                                                             angle=angle)
            x_train.append(_x_train)
            y_train.append(_y_train)
            x_val.append(_x_val)
            y_val.append(_y_val)
            x_test.append(_x_test)
            y_test.append(_y_test)
        model = ArchModel()
        
    
        rs = np.random.RandomState(42+k)
        
        opt = torch.optim.SGD([{'params': model.rots.parameters(), 'lr': lr0},
                              {'params': model.models.parameters(), 'lr': lr}],
                              lr=lr)
                              
                              
        opt2 = torch.optim.SGD(model.gammas, lr=lr2)

        crit = torch.nn.MSELoss()
        tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS
        losses = []
        losses2 = []
        for e in tq:
            
            
            for d in range(D):
                opt2.zero_grad()
                gammas = torch.nn.functional.softmax(model.gammas[d])
                m = torch.distributions.Categorical(gammas)
                
            
                x = torch.tensor(x_val[d]).float()
                y = torch.tensor(y_val[d]).float()
                gamma_id = m.sample((x.shape[0],))
                log_prob = m.log_prob(gamma_id)
                onehot = torch.nn.functional.one_hot(gamma_id, HEAD_NUM)
            
            
                
                out = model(x, d, gammas = onehot, use_softmax = False)[:,0]
                loss2 = crit(out, y) 
                
                opt2.zero_grad()
                reward = -loss2 # more -> better
                loss_to_min = - (reward * log_prob) 
                loss_to_min.sum().backward()
                torch.nn.utils.clip_grad_value_(model.gammas, GRAD_CLIP)
                opt2.step()

                            
                opt.zero_grad()
                x = torch.tensor(x_train[d]).float()
                y = torch.tensor(y_train[d]).float()
                gamma_id = m.sample((x.shape[0],))
                log_prob = m.log_prob(gamma_id)
                onehot = torch.nn.functional.one_hot(gamma_id, HEAD_NUM)
            
                
                out = model(x, d, gammas = onehot, use_softmax = False)[:,0]
                loss = crit(out, y)
                
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP)
                opt.step()
                losses.append(loss.item())
                losses2.append(loss2.item())
            
            tq.set_description(str(np.mean(losses))+';'+str(np.mean(losses2)))

        tq = tqdm.tqdm_notebook(range(EPOCH_NUM))
        
        # search step. Now we concatenate train and validation
        x_train = np.concatenate([x_train, x_val], axis=1)
        y_train =  np.concatenate([y_train, y_val], axis=1)
        
        
        for g in model.gammas:
            print (g.argmax(1))
        
        for g in model.gammas:
            print (g)
        
        opt = torch.optim.SGD([{'params': model.rots.parameters(), 'lr': lr0},
                              {'params': model.models.parameters(), 'lr': lr}],
                              lr=lr)
        
        for e in tq:

            for d in range(D):
                loss = 0.0
                opt.zero_grad()

                d2 = rs.choice([i for i in range(D) if i != d])

                x = torch.tensor(x_train[d]).float()
                y = torch.tensor(y_train[d]).float()
                out = model(x, d, use_softmax=False)[:,0]

               
                loss = crit(out, y) 
                
                
                loss.backward()
                torch.nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP)
                opt.step()
            angles= [str(r.angle.item()) for r in model.rots]
            #print (angles)
            tq.set_description(' '.join(angles))
        total = 0
        for d in range(D):

            out = model(torch.tensor(x_test[d]).float(), d, use_softmax=False)
            total += crit2(out[:,0], torch.tensor(y_test[d]).float()).item()
        
        acc = total/D
        accs.append(acc)
        param_num.append(calc_params(model.gammas))
        print (acc, param_num[-1])
        
    with open(fname+'.txt', 'a') as out:
        out.write(f'D={D}, HEAD_NUM={HEAD_NUM}, RUN_NUM={RUN_NUM}, CRIT={np.mean(accs)}+-{np.std(accs)}.  MIN={np.min(accs)}, MAX={np.max(accs)}')
        out.write(f' PARAMS={np.mean(param_num)}+-{np.std(param_num)}. MIN={np.min(param_num)}, MAX={np.max(param_num)}\n')
    
    with open(fname+'.jsonl', 'a') as out:
        out.write(json.dumps({'D': D, 'HEAD_NUM': HEAD_NUM, 'RUN_NUM': RUN_NUM, 'accs': accs, 'params': param_num})+'\n')
    print (model.gammas)
    for r in model.rots:
        print (r.angle)
    return (np.mean(accs), np.mean(param_num))


In [15]:
# helper for make inference for a model

for D in [2,3,4]:
    for HEAD_NUM in [D, 8]:
        rl("rl")

<ipython-input-14-77ef0e35c913>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM )) # taking half of epoch num for NAS


  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-14-77ef0e35c913>:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  gammas = torch.nn.functional.softmax(model.gammas[d])
<ipython-input-14-77ef0e35c913>:82: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm.tqdm_notebook(range(EPOCH_NUM))


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([0, 1])
tensor([1, 1])
Parameter containing:
tensor([[1.0206, 0.7767],
        [0.0293, 1.3129]], requires_grad=True)
Parameter containing:
tensor([[ 0.4217,  0.5696],
        [-0.2292,  1.2794]], requires_grad=True)
1.55710107088089 11


...

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([4, 3])
tensor([6, 4])
tensor([4, 5])
tensor([2, 4])
Parameter containing:
tensor([[-0.2450,  0.2589,  0.7618,  0.5315,  0.9201,  0.7319,  0.5521,  0.7595],
        [-0.2614,  0.7526,  0.5243,  0.7707,  0.5784,  0.6437,  0.7142,  0.2572]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.4618,  0.0492,  0.8811,  0.8833,  0.0243,  0.8038,  1.1544,  0.3182],
        [ 0.6853,  0.2737,  0.0677,  0.1280,  0.9281,  0.2208, -0.3089,  0.8706]],
       requires_grad=True)
Parameter containing:
tensor([[ 1.4823,  0.1989,  0.0178, -0.2912,  3.1611, -0.8395,  0.1740,  0.3139],
        [ 1.1259,  0.7875, -0.0514,  0.6762, -0.3634,  2.8317,  1.9746, -0.9903]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.2102,  0.7942,  0.8442,  0.2588,  0.1297,  0.6320,  0.5955,  0.2078],
        [ 0.3477,  0.2838,  0.4341,  0.1303,  0.9156,  0.6080,  0.1817, -0.0703]],
       requires_grad=True)
0.8680248707532883 30


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([6, 2])
tensor([0, 7])
tensor([0, 5])
tensor([7, 6])
Parameter containing:
tensor([[-0.0407,  0.7823,  0.4713,  0.6215,  0.6140,  0.8036,  0.9535,  0.9182],
        [ 0.0551,  0.9357,  0.9961,  0.6249,  0.3865,  0.5871,  0.1694,  0.0634]],
       requires_grad=True)
Parameter containing:
tensor([[ 1.1341,  0.9073,  0.3252,  0.5982, -0.0440,  1.0708,  0.6163, -0.3422],
        [ 0.5999,  0.7295,  0.9327,  0.6321,  0.6991, -0.1990,  0.7195,  0.9493]],
       requires_grad=True)
Parameter containing:
tensor([[1.0334, 0.6726, 0.9499, 0.3013, 0.0438, 0.7300, 0.3600, 0.7107],
        [0.3030, 0.8018, 0.0827, 0.1560, 0.3716, 1.0227, 0.0515, 0.5381]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.4631,  0.5475,  0.4734,  0.1090,  0.2091,  0.2701,  0.2372,  0.6350],
        [ 0.5768,  0.8937,  0.6198,  0.4600,  0.7633, -0.0101,  0.9276,  0.5441]],
       requires_grad=True)
1.0500913262367249 39
[Parameter containing:
tensor([[-0.0407,  0.7823,  0.4713,  0.6215,  0.6140,  